In [ ]:
from labeyrieClasses import target #, fftw_psd
import numpy as np
import matplotlib.pyplot as plt
import ctypes

%matplotlib inline

In [ ]:
# Import fits data
binary = target()
binary.fits.fileName = "/home/niels/Documents/FITS/KP330.fits"
binary.fits.read(numDimensions = 3)

# Import shared C library
fftw_psd_dll = ctypes.CDLL('/home/niels/Dropbox/Thesis/Python/dev/fftw_psd.so')

# Image sizing parameters
imgsize = 512
psd_n = imgsize*(int(imgsize/2)+1)
img_n = imgsize**2

In [ ]:
# FFTW PSD Function Declaration
def fftw_psd(input_img):
    # Reshape Square array to be flat
    input_img_flat = np.reshape(input_img.astype(np.float32),(imgsize**2,1))

    # Create pointers for in/out
    img_ptr = (input_img_flat).ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    out_ptr = (np.zeros(img_n,np.float32)).ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    # Array type to be passed to wrapped C function
    # Set input argument to be flat array of doubles (# of input img pixels)
    fftw_psd_dll.psd.argtypes = [ctypes.POINTER(ctypes.c_float)]
    fftw_psd_dll.psd.restype = ctypes.POINTER(ctypes.c_float * psd_n)

    # Calculate PSD, get a pointer returned
    out_ptr = fftw_psd_dll.psd(img_ptr)

    # Reshape array to image
    psd_image = np.reshape(out_ptr.contents,(imgsize,int(imgsize/2+1)))
    
    # Return PSD Image
    return psd_image

In [ ]:
# Calculate Average using NP and FFTW function
fftw_psd_img = np.zeros((imgsize,int(imgsize/2+1)))
np_psd_img = np.zeros((imgsize,imgsize))

for image in binary.fits.data:
    #plt.imshow(image)
    fftw_psd_img += fftw_psd(image)
    np_psd_img += (np.abs(np.fft.fft2(image))**2)

In [ ]:
# Compare PSD images from both FFTW and NP
np_psd_img_shifted = np.fft.fftshift(np_psd_img)

plt.figure(figsize=(20,10))
plt.subplot(1,2,1)
plt.imshow(np.log10(fftw_psd_img))
plt.title("fftw PSD")
plt.subplot(1,2,2)
plt.imshow(np.log10(np_psd_img_shifted))
plt.title("numpy fft psd")

In [ ]:
# Looking for symmetry in PSD images
indices = ((0,0), (1,1), (511,511), (510,510))

for index in indices:
    print("index = ", index, " \t np_psd_img = ", np_psd_img[index], "\t np_psd_img_shifted = ", np_psd_img_shifted[index])
    
# We see symmetry at (1,1) and (511,511)

In [ ]:
# Looking for FFTshift pattern
indices = ((256,256), (255,255), (257,257))

for index in indices:
    print("Index[", index, "] = ", np_psd_img_shifted[index])

In [ ]:
imgsize = 8
fftw_psd_img_complete = np.zeros((imgsize,imgsize))

xx,yy = np.meshgrid(np.arange(imgsize),np.arange(imgsize))
xx = np.subtract(xx,int(imgsize/2))

print(xx)
print(yy)